# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
#import warnings
#warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [3]:
# Load necessary packages

In [4]:
# remove "object"-type features and SalesPrice from `X`
X = df.select_dtypes(exclude='object').copy()
X.drop(['SalePrice'], axis=1, inplace=True)

# Impute null values
# for each column, calc median and use to fillna
for col in X.columns:
    med = X[col].median()
    X[col].fillna(value=med, inplace=True)

# Create y
y = df['SalePrice'].copy()

In [5]:
y.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

Look at the information of `X` again

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [7]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import numpy as np

In [8]:
np.random.seed(8675309)

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
# Fit the model and print R2 and MSE for train and test
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [10]:
train_r2 = lr.score(X_train, y_train)
test_r2 = lr.score(X_test, y_test)
train_mse = mean_squared_error(y_train, lr.predict(X_train))
test_mse = mean_squared_error(y_test, lr.predict(X_test))

In [11]:
print("Train\nR2 : {:.3f}\nMSE :${:,.0f}".format(train_r2, train_mse)) #${:,.2f}
print("Test\nR2 : {:.3f}\nMSE :${:,.0f}".format(test_r2, test_mse))

Train
R2 : 0.823
MSE :$1,056,138,397
Test
R2 : 0.776
MSE :$1,640,168,131


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [12]:
from sklearn import preprocessing

In [13]:
# Scale the data and perform train test split
sc = preprocessing.StandardScaler()
sc.fit(X_train)

X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/opt/conda/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


Perform the same linear regression on this data and print out R-squared and MSE.

In [14]:
# Fit the model and print R2 and MSE for train and test
lr_sc = LinearRegression()
lr_sc.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
train_sc_r2 = lr_sc.score(X_train_scaled, y_train)
test_sc_r2 = lr_sc.score(X_test_scaled, y_test)
train_sc_mse = mean_squared_error(y_train, lr_sc.predict(X_train_scaled))
test_sc_mse = mean_squared_error(y_test, lr_sc.predict(X_test_scaled))

In [16]:
print("Train\nR2 : {:.3f}\nMSE :${:,.0f}".format(train_sc_r2, train_sc_mse)) #${:,.2f}
print("Test\nR2 : {:.3f}\nMSE :${:,.0f}".format(test_sc_r2, test_sc_mse))

Train
R2 : 0.823
MSE :$1,056,304,708
Test
R2 : 0.776
MSE :$1,639,533,936


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

To make the downstream scaling and dummy-fing easier. I'm going to redo the train-test split on the whole original dataframe.

In [28]:
df.shape

(1460, 81)

In [31]:
# Split the datasets
X_train, X_test, y_train, y_test = train_test_split(df.drop(['SalePrice'], axis=1), df['SalePrice'])

In [32]:
# Check row counts
X_train.shape

(1095, 80)

In [33]:
# Breakup into continuous and categorical columns
X_cont_train = X_train.select_dtypes(exclude='object').copy()
X_cat_train = X_train.select_dtypes(include='object').copy()

X_cont_test = X_test.select_dtypes(exclude='object').copy()
X_cat_test = X_test.select_dtypes(include='object').copy()


In [ ]:
# Run scaling for continous features
# Create object
scaler = preprocessing.StandardScaler()

# Fit to training data
scaler.fit(X_cont_train)

# Scale train and test data
X_sc_train = scaler.transform(X_cont_train)
X_sc_test = scaler.transform(X_cont_test)


In [35]:
# Get categorical dummies

Xcctr = pd.get_dummies(X_cat_train)

In [37]:
Xcctr.head()
Xcctr.shape

(1095, 247)

In [38]:
Xccte = pd.get_dummies(X_cat_test)

In [ ]:
Xcct

In [21]:
# Make dummies
X_cat = pd.get_dummies(X_cat)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [22]:
# perform train|test split on X_cat, should be same indices because fixed seed.
X_cat_train, X_cat_test = train_test_split(X_cat)

In [23]:
X_cat_train.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1440,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
823,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
790,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
112,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
182,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [25]:
X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
522,523,50,50.0,5000,6,7,1947,1950,0.0,399,...,420,0,24,36,0,0,0,0,10,2006
1357,1358,20,69.0,12537,5,6,1971,2008,0.0,734,...,500,0,0,0,0,0,0,0,4,2010
28,29,20,47.0,16321,5,6,1957,1997,0.0,1277,...,319,288,258,0,0,0,0,0,12,2006
939,940,70,69.0,24090,7,7,1940,1950,0.0,0,...,349,56,0,318,0,0,0,0,6,2010
988,989,60,69.0,12046,6,6,1976,1976,298.0,156,...,551,0,224,0,0,0,0,0,6,2007


In [ ]:
# Your code here
X_all = pd.concat([])

Perform the same linear regression on this data and print out R-squared and MSE.

In [ ]:
# Your code here

Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [ ]:
# Your code here

With a higher regularization parameter (alpha = 10)

In [ ]:
# Your code here

## Ridge

With default parameter (alpha = 1)

In [ ]:
# Your code here

With default parameter (alpha = 10)

In [ ]:
# Your code here

## Look at the metrics, what are your main conclusions?

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [ ]:
# number of Ridge params almost zero

In [ ]:
# number of Lasso params almost zero

Compare with the total length of the parameter space and draw conclusions!

In [ ]:
# your code here

## Summary

Great! You now know how to perform Lasso and Ridge regression.